In [2]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime as dt
import yfinance as yf
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import statsmodels.api as sm

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [34]:
from fredapi import Fred
fred = Fred(api_key = 'ec682064ba501918755568565884325e')
start_date = ('01/01/2000')
end_date = ('12/31/2017')
end_date2 = ('12/31/2022')

eco_unc = pd.Series((fred.get_series('WLEMUINDXD', start_date, end_date)), name = 'eco_unc')
tbill = pd.Series((fred.get_series('DTB3', start_date, end_date)), name = 'tbill')
wti = pd.Series((fred.get_series('DCOILWTICO', start_date, end_date)), name = 'wti')
vix = pd.Series((fred.get_series('VXVCLS', start_date, end_date)), name = 'vix')
ccsa = pd.Series((fred.get_series('CCSA', start_date, end_date)), name = 'unemp')


eco_unc2 = pd.Series((fred.get_series('WLEMUINDXD', start_date, end_date2)), name = 'eco_unc')
tbill2 = pd.Series((fred.get_series('DTB3', start_date, end_date2)), name = 'tbill')
wti2 = pd.Series((fred.get_series('DCOILWTICO', start_date, end_date2)), name = 'wti')
vix2 = pd.Series((fred.get_series('VXVCLS', start_date, end_date2)), name = 'vix')
ccsa2 = pd.Series((fred.get_series('CCSA', start_date, end_date2)), name = 'unemp')

len(eco_unc)


6575

In [4]:
#converting weekly data to daily
unem = ccsa.to_frame(name="unem")
unem.reset_index(inplace=True)
pd.to_datetime(unem['index'],format='%m/%d/%Y')
unem = unem.set_index('index').resample('D').ffill().reset_index()
unem = unem.set_index('index')
unemp = unem.squeeze()


unem2 = ccsa2.to_frame(name="unem")
unem2.reset_index(inplace=True)
pd.to_datetime(unem2['index'],format='%m/%d/%Y')
unem2 = unem2.set_index('index').resample('D').ffill().reset_index()
unem2 = unem2.set_index('index')
unemp2 = unem2.squeeze()


In [5]:
data = pd.concat([tbill, wti, vix, unemp], axis =1).dropna()
data2 = pd.concat([tbill2, wti2, vix2, unemp2], axis =1).dropna()


In [33]:
st_date = dt.datetime(2000, 1, 1)
ed_date = dt.datetime(2017, 12, 31)
SNP500 = yf.download('^GSPC', st_date, ed_date)
SNP500 = SNP500['Adj Close'].to_frame()
# SNP500.fillna(method = 'bfill', inplace=True)
SNP500['SMA 5'] = SNP500['Adj Close'].rolling(5).mean()
SNP500['SMA 200'] = SNP500['Adj Close'].rolling(200).mean()
conditions = [SNP500['SMA 5'] >= SNP500['SMA 200'],
              SNP500['SMA 5'] < SNP500['SMA 200']]
choices = [1, 0]
SNP500['outcome'] = np.select(conditions, choices, 0)
SNP500 = SNP500['outcome'].dropna()
SNP500

[*********************100%***********************]  1 of 1 completed


Date
2000-01-03    0
2000-01-04    0
2000-01-05    0
2000-01-06    0
2000-01-07    0
             ..
2017-12-22    1
2017-12-26    1
2017-12-27    1
2017-12-28    1
2017-12-29    1
Name: outcome, Length: 4528, dtype: int64

In [9]:
features = pd.merge(data, SNP500, left_index = True, right_index = True).dropna()
X = features[['tbill', 'wti', 'vix', 'unem']] 
X2 = data2[['tbill', 'wti', 'vix', 'unem']]
y = features['outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=101)


In [21]:
len(y)


2517

In [22]:
len(SNP500)

4528

In [10]:
data.corr()

,tbill,wti,vix,unem
tbill,1.000000,0.114596,-0.039390,-0.225603
wti,0.114596,1.000000,-0.100543,0.173749
vix,-0.039390,-0.100543,1.000000,0.627377
unem,-0.225603,0.173749,0.627377,1.000000


In [11]:
vif = pd.DataFrame()
vif["ft"] = data.columns
vif["VIF"] = [variance_inflation_factor(data.values, i)
                         for i in range(len(data.columns))]
vif

,ft,VIF
0,tbill,1.513578
1,wti,7.137409
2,vix,11.826841
3,unem,17.528333


In [13]:
log_regression = LogisticRegression()
model = log_regression.fit(X_train,y_train)

y_pred = log_regression.predict(X_test)

print('score', log_regression.score(X_test, y_test))   #Return the mean accuracy on the given test data and labels.

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

score 0.716931216931217
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       214
           1       0.72      1.00      0.84       542

    accuracy                           0.72       756
   macro avg       0.36      0.50      0.42       756
weighted avg       0.51      0.72      0.60       756



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [14]:
model.coef_

array([[-2.01805404e-13,  2.16095141e-11, -5.06603458e-12,
         2.50099624e-07]])

In [15]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[  0, 214],
       [  0, 542]])

In [16]:
log_reg = sm.Logit(y_train, X_train).fit(cov_type = 'HC1')
#result = model1.fit()
yhat = log_reg.predict(X2)
log_reg.summary()

Optimization terminated successfully.
         Current function value: 0.408264
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                outcome   No. Observations:                 1761
Model:                          Logit   Df Residuals:                     1757
Method:                           MLE   Df Model:                            3
Date:                Sun, 20 Feb 2022   Pseudo R-squ.:                  0.2839
Time:                        07:52:29   Log-Likelihood:                -718.95
converged:                       True   LL-Null:                       -1004.0
Covariance Type:                  HC1   LLR p-value:                2.961e-123
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
tbill         -0.8944      0.119     -7.543      0.000      -1.127      -0.662
wti            0.0346      0.003     11.830      0.000       0.029       0.040
vix           -0.2365      0.014    -17.458      0.000      -0.263      -0.210
unem        1.179e-06   1.21e-07      9.760      0.000    9.43e-07    1.42e-06
==============================================================================
"""

In [17]:
prediction = list(map(round, yhat))
prediction

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [25]:
SPX = yf.download('^GSPC', start=st_date, end=ed_date)
RUA = yf.download('^RUA', start=st_date, end=ed_date)
# price is already adjusted
SPX = SPX.drop(['Adj Close'], axis=1)
RUA = RUA.drop(['Adj Close'], axis=1)
# 10/200 MA
SPX['MA200'] = SPX['Close'].rolling(window=200).mean()
RUA['MA200'] = RUA['Close'].rolling(window=200).mean()
SPX['MA10'] = SPX['Close'].rolling(window=10).mean()
RUA['MA10'] = RUA['Close'].rolling(window=10).mean()
# ATR 10 for volatility
# SPX['ATR10'] = atr(high=SPX['High'], low=SPX['Low'], close=SPX['Close'], window=10)
# RUA['ATR10'] = atr(high=RUA['High'], low=RUA['Low'], close=RUA['Close'], window=10)
# Daily returns
SPX['Return'] = SPX['Close'].pct_change()
RUA['Return'] = RUA['Close'].pct_change()
# StDev of daily returns
SPX['StDev'] = SPX['Return'].rolling(10).std()
RUA['StDev'] = RUA['Return'].rolling(10).std()
# Side column if MA10 > MA200 1 (long), else -1 (short)
SPX['Side'] = np.where(SPX['MA10'] > SPX['MA200'], 1, -1) 
RUA['Side'] = np.where(RUA['MA10'] > RUA['MA200'], 1, -1)
# Shift side column forward by 1 so that we don't commit lookahead bias
SPX['Side'] = SPX['Side'].shift(1)
RUA['Side'] = RUA['Side'].shift(1)

SPX['StratReturns'] = 0
RUA['StratRetiuns'] = 0

# drop values for first 200 days since no MA200
# SPX.dropna(inplace=True)
# RUA.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [29]:
len(prediction)
# len(RUA)

3535

In [28]:
len(SNP500)

4528

In [30]:
len(yhat)

3535